In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

# Dataset

In [ ]:
train_dataset = load_dataset("AWeirdDev/zh-tw-recipes-sm", split="train")

In [ ]:
def formatting_func(example):
    output_texts = [f"# 標題\n{example['title'][i]}\n\n# 腳步\n{example['steps'][i]}" for i in range(len(example))]
    return output_texts

# Tokenizer and Model

In [ ]:
tokenizer_checkpoint = "uer/gpt2-chinese-cluecorpussmall"
model_checkpoint = "uer/gpt2-chinese-cluecorpussmall"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto")

if tokenizer.pad_token is None and model.config.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Train

## Config

In [ ]:
args = SFTConfig(output_dir="tmp_trainer/zh_tw_recipes_sm")

## Run

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func
)
trainer.train()